In [1]:
pip install accelerate==1.2.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install faiss-gpu==1.7.2 huggingface-hub==0.26.3 ipywidgets==8.1.5 jupyterlab==4.3.2 langchain==0.3.9 langchain-community==0.3.9

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain-huggingface==0.1.2 numpy==1.26.4 pydantic==2.10.2 sentence-transformers==3.3.1 torch==2.5.1 transformers==4.47.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
pip install flask flask-cors

Note: you may need to restart the kernel to use updated packages.


## Vector Store үүсгэх

RAG системийн үндсэн хоёр хэсгийн нэг, заавал бэлдсэн байх ёстой зүйл бол өгөгдлөө embedding буюу тоон хэлбэрт шилжүүлж, vector store-д хадгалсан байх юм.

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*vAvDBIbr8MnL_Q51mBtBhw.png)

### Өгөгдөл унших

`data` хавтсанд байрлах `csv` файлыг уншиж LangChain [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)-н жагсаалт болгоно.

- `khas_bank_news.csv`
- `khas_bank_pages.csv`
- `khas_bank_products.csv`

In [6]:
DATA_DIR = "../../data/"
loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_news.csv",
                   source_column="link",
                   metadata_columns=["title", "date", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["title", "link", "date", "content"],
                       }
                   )
news_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_products.csv",
                   source_column="link",
                   metadata_columns=["id", "side_menu_text", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["id", "content", "side_menu_text", "link"],
                       }
                   )
product_data = loader.load()[1:]

loader = CSVLoader(file_path=f"{DATA_DIR}khas_bank_pages.csv",
                   source_column="link",
                   metadata_columns=["title", "link"],
                   content_columns=["content"],
                   csv_args={
                       "fieldnames": ["title", "content", "link"],
                       }
                   )
pages_data = loader.load()[1:]

all_data = news_data + product_data + pages_data
print(len(news_data), len(product_data), len(pages_data))
print(f"Нийт хуудасны тоо: {len(all_data)}")
print(all_data[0])

664 71 50
Нийт хуудасны тоо: 785
page_content='content: ХасБанк 2024 оны III улиралд 43.4 тэрбум төгрөгийн татварын дараах цэвэр ашигтай ажиллалааХасБанк 2024 оны III улиралд 43.4 тэрбум төгрөгийн (өмнөх улирлаас +0.4%) татварын дараах цэвэр ашигтай ажиллалаа. Ингэснээр жилийн өссөн дүнгээр татварын дараах цэвэр ашиг 128.7 тэрбум төгрөгт (өмнөх жилийн мөн үеэс +26.7%) хүрч, өөрийн хөрөнгийн өгөөж (annualized ROE) 26.2%,нэгж хувьцаанд ногдох цэвэр ашиг (EPS) 122.3 төгрөгтхүрлээ2024 оны III улирлын санхүүгийн үзүүлэлтээс онцолбол:Зээл ба санхүүгийн түрээсийн багц 3,609 тэрбум төгрөгт хүрч, өмнөх улирлаас 8.6%, оны эхнээс 31.2% өссөн нь III улирлын цэвэр хүүгийн орлогыг өмнөх улирлаас 9.0% өсөхөд нөлөөлсөн байна.Активын чанарын үзүүлэлт тогтвортой сайн хэвээр байгаа бөгөөд нийт зээл болон санхүүгийн түрээсийн багцад эзлэх чанаргүй зээлийг 2.2%-д удирдан ажиллаж байна.Харилцах, хадгаламжийн эх үүсвэр өмнөх улирлаас 0.1%-аар, оны эхнээс 11.3%-аар өсөж, 3,202 тэрбум төгрөгт хүрлээ.2024 оны э

### Өгөгдөл хуваах

Document хэт урт бол vector store-с хайх хайлтын чанар муудах магадлалтай тул доорх хэсэгт агуулгыг тэмдэгтийн тоогоор таслан хэд хэдэн Document болгов.

Жич: RAG-н чанарыг сайжруулахын тулд зөвхөн тэмдэгтийн тоогоор хуваахаас гадна өгүүлбэр, цогцолбор, бүлэг гэх мэтчилэн өгөгдөлд тохирсон арга олох нь чухал.

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100
)
docs = text_splitter.split_documents(documents=all_data)

### Embedding

Embedding нь энгийн үгээр ямар нэгэн өгөгдлийг вектор дүрслэл рүү хөрвүүлсэн хэлбэр бөгөөд тухайн вектор нь олон хэмжээст огторгуйд ижил, төстэй утга бүхий өгөгдөлтэй ойр, эсрэг утгатай өгөгдлөөс хол байрладаг.

Энэхүү жишээ RAG-д [`sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) загварыг сонгон авлаа.

![embedding model](https://python.langchain.com/assets/images/embeddings_concept-975a9aaba52de05b457a1aeff9a7393a.png)

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

### Vector Store

Харин vector store нь embedding болон өгөгдөл эсвэл түүний заагчийг хамтад нь хадгалж, хайлтын query орж ирэхэд мөн түүнийг вектор хэлбэрт хөрвүүлэн, санд байх векторуудаас хамгийн ойрыг хайдаг өгөгдлийн сан юм.

Энэхүү жишээнд FAISS vector store сонгож ашиглалаа.

![vectorestore](https://python.langchain.com/assets/images/vectorstores-2540b4bc355b966c99b0f02cfdddb273.png)

In [9]:
# Бүх document-с embedding үүсгэж, vector store-д хадгалах
# (өмнө нь үүсгэсэн бол заавал дахин үүсгэх шаардлагагүй, доорх load_local функцээр хадгалсан файлаас уншиж болно)
vector_store = FAISS.from_documents(docs, embeddings)
vector_store.save_local("faiss_index")

In [10]:
# Үүсгэсэн vector store-г файлаас унших
vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

Vector Store үүсгэсэн бол түүнээс ойролцоо утгатай өгөгдөл хайж үзье.

![retriever](https://python.langchain.com/assets/images/retriever_concept-1093f15a8f63ddb90bd23decbd249ea5.png)

## Өгөгдөл хайх, хариулах

Өмнөх хэсэгт өгөгдлөө бэлтгэж, vector store-т хадгалсан бол энэ хэсэгт хэрэглэгчийн асуусан асуултаар vector store-с хайлт хийж, LLM-р хариулт өгөх болно.

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*8dUBh25peTpsucMpL1ypUg.png)

### LLM

Хайж олсон мэдээлэл, хэрэглэгчийн асуултад тохирох хариулт үүсгэхийн тулд бичвэр үүсгэгч Large Language Model ашиглана.

Энд Meta компанийн сургасан [8 тэрбум параметртай Llama 3.1(https://huggingface.co/meta-llama/Llama-3.1-8B)] загварыг сонгов. (Hugging Face дээр загвар ашиглах зөвшөөрөл хүссэн маягт бөглөх ёстой)


### Chain: Retrieval -> Prompt -> LLM

Prompt-н `{context}`-н оронд хайж олсон мэдээлэл, `{input}`-н оронд хэрэглэгчийн асуусан асуулт орох болно.

In [14]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the user question. If you don't know the answer to the question, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    'Context:\n"""\n{context}\n"""'
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "Question: \"\"\"{input}\"\"\""),
        ("assistant", "Answer: "),
    ]
)

# vector store-с document хайгч, k параметраар хамгийн ойр утгатай хэдэн document буцаахыг дамжуулна
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Chain үүсгэх
# input -> retrieval -> prompt -> llm -> answer
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/root/rag-starter/src/api']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [361431] using WatchFiles
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:18<00:00,  4.56s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
